In [ ]:
"""
This code only extracts the Landslide data from 2000 to 2019 from the Global Landslide Catalog: NASA Goddard (1970-2019) product, without altering the original spatial resolution of the data. For subsequent analysis, see Dataprepare-Disasters.R.

Script Functionality:
This script uses Google Earth Engine to filter the Global Landslide Catalog data for the years 2000 to 2019. The filtered dataset is then exported as a CSV file to Google Drive for further analysis.

Input Parameters:
Global Landslide Catalog dataset from Earth Engine for landslide events from 1970-2019.

Dependencies:
Earth Engine Python API: pip install earthengine-api
Geemap library for exporting data: pip install geemap
Version Information:

Version: v1.0
Change Log: October 30, 2024.
"""

import ee
import geemap

# Initialize Earth Engine
ee.Authenticate()  # Authenticate Earth Engine (requires Google account login)
ee.Initialize(project="lizeshuo")  # Initialize Earth Engine with specific project ID

# Load the Global Landslide Catalog dataset and filter it to events from 2000-2019
glc = ee.FeatureCollection("projects/sat-io/open-datasets/events/global_landslide_1970-2019")

# Export the filtered dataset to a CSV file in Google Drive
geemap.ee_export_vector_to_drive(glc, description='Landslide_GLC_2000_2019', fileFormat='CSV')

print("Data has been exported to Google Drive. Please check your Drive.")
# End of the script


In [ ]:
"""

This code only extracts the Landslide data from 2000 to 2019 from the Global Landslide Catalog: NASA Goddard (1970-2019) product, without altering the original spatial resolution of the data. For subsequent analysis, see Dataprepare-Disasters.R.

Script Functionality:
This script reads a CSV file containing landslide data, converts the 'event_date' column to a date format, and filters for records from the years 2000 to 2019. The filtered data is then saved to a new CSV file for further use.

Input Parameters:
File path to the original landslide data CSV file (update file_path with the local path).

Dependencies:
Pandas: pip install pandas

Version Information:
Version: v1.0
Change Log: October 30, 2024.
"""
import pandas as pd

# Read the CSV file
file_path = 'E:/ghana/landslide/landslide_data_1970_2019.csv'  # Replace with your file path
data = pd.read_csv(file_path)  # Load the landslide data from CSV into a DataFrame

# Convert the 'event_date' column to date format
data['event_date'] = pd.to_datetime(data['event_date'], errors='coerce')  # Ensure 'event_date' is in datetime format

# Filter data between the years 2000 and 2019
filtered_data = data[(data['event_date'].dt.year >= 2000) & (data['event_date'].dt.year <= 2019)]  # Select events from 2000 to 2019

# Save the filtered results to a new CSV file
filtered_file_path = 'F:/[2024.07]Ghana/Processed Data/Disasters/Extraction/Landslide_GLC_2000_2019.csv'
filtered_data.to_csv(filtered_file_path, index=False)  # Export filtered data to specified CSV path without index

print(f"Extraction complete. Results saved at {filtered_file_path}")
# End of the script


In [ ]:
"""
This code only extracts the Landslide data from 2000 to 2019 from the Global Landslide Catalog: NASA Goddard (1970-2019) product, without altering the original spatial resolution of the data. For subsequent analysis, see Dataprepare-Disasters.R.

Script Functionality:
This script processes a CSV file containing landslide event data from 2000 to 2019, 
preparing it for spatial analysis by converting it to a Shapefile format. 
Key steps include renaming columns to ensure compatibility, truncating long text fields, creating geometries from coordinate data, and setting the coordinate reference system to WGS84 (EPSG:4326).

Input Parameters:
File path to the CSV file containing the landslide data (update file_path with the local path).
Dependencies:

Pandas: pip install pandas
GeoPandas: pip install geopandas
Shapely (installed with GeoPandas for geometry handling)

Version Information:
Version: v1.0
Change Log: October 30, 2024.
"""

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the CSV file
file_path = 'F:/[2024.07]Ghana/Processed Data/Disasters/Extraction/Landslide_GLC_2000_2019.csv'
data = pd.read_csv(file_path)  # Load data from CSV into a DataFrame

# Rename columns to ensure field names are under 10 characters
data.rename(columns={
    'system:index': 'sys_idx',
    'admin_di_1': 'admin_id',
    'event_desc': 'desc',
    'event_impo': 'impact',
    'location_a': 'loc_area',
    'location_d': 'loc_desc',
    'source_lin': 'src_link',
    'source_nam': 'src_name',
    'submitted_': 'submitted'
}, inplace=True)

# Truncate long text fields to limit text length
data['desc'] = data['desc'].astype(str).str.slice(0, 254)
data['src_link'] = data['src_link'].astype(str).str.slice(0, 254)
data['loc_desc'] = data['loc_desc'].astype(str).str.slice(0, 254)

# Drop unnecessary '.geo' field
data.drop(columns=['.geo'], inplace=True)

# Create a geometry field using latitude and longitude
data['geometry'] = data.apply(
    lambda row: Point(row['longitude'], row['latitude']) if pd.notnull(row['longitude']) and pd.notnull(row['latitude']) else None,
    axis=1
)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')

# Set the coordinate reference system (WGS84 - EPSG:4326)
gdf.set_crs(epsg=4326, inplace=True)

# Check and adjust data types for compatibility
for col in gdf.columns:
    if gdf[col].dtype == 'object':
        gdf[col] = gdf[col].astype(str)
    elif gdf[col].dtype == 'float64':
        gdf[col] = gdf[col].astype('float32')

# Save the GeoDataFrame as a Shapefile
shapefile_path = 'F:/[2024.07]Ghana/Processed Data/Disasters/Extraction/Landslide_GLC_2000_2019.shp'
gdf.to_file(shapefile_path, driver='ESRI Shapefile')

print(f"Conversion complete. Shapefile saved as {shapefile_path}")
# End of the script

转换完成，Shapefile已保存为 E:/ghana/landslide/shp/landslide_data_2000_2019.shp


d:\anaconda3\envs\gee\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 'Thanks to the WSB’er who shared the photo; after a night of pounding rain, a mudslide behind a building in the 1300 block of Alki Avenue SW is reported to have damaged three vehicles, pushing them into each other. We’ll be checking to find out more;�' of field desc has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(
